In [ ]:
import torch
from transformers import AutoTokenizer,AutoModelForCausalLM


path='/root/model/Shanghai_AI_Laboratory/internlm-chat-7b'
tokenizer = AutoTokenizer.from_pretrained(path,trust_remote_code=True)
model=AutoModelForCausalLM.from_pretrained(path,trust_remote_code=True,torch_dtype=torch.bfloat16,device_map='auto')
model.eval()

In [ ]:
inputs=tokenizer("你是谁",return_tensors='pt')
inputs={k:v.to('cuda') for k,v in inputs.items() if torch.is_tensor(v)}
#inputs={k: v.to(self.device) for k, v in inputs.items() if torch.is_tensor(v)}
outputs=model(**inputs)
outputs

In [75]:
outputs['logits'].shape

torch.Size([1, 3, 103168])

In [57]:
max_token=torch.max(outputs['logits'],1)
tokenizer.batch_decode(max_token[0], skip_special_tokens=True)

['ntnt00ntntntntntntntntntnt Mntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntntavntntntntntntntntntntntntntntntntntntntntntntntntntntntntntnt00ourour00ntour00our0000ourour000000000000000000our00000000our000000000000nt00000000000000000000000000000000000000000000our0000our00000000ntntptntly               ntavlyavntntntntavavavavntavntavavntavavavntntavntavptourourptourourourourourour0000nt00ourourptntntntavntntntntntntntntntntntour00our if00our00our0000our00ourourourour00our00pt0000pt0000ptpt00ourourourourour00ourourptourour ifourourptptourourourourourptourourourptourptourourptourpt00ourourourourour0000 ifourptourourour ifourptourourourourourptourour Bptourptourptptptouravntavntavntavav00our Bntavptavavavavavavouravavavavavlyavntntavavavptpt0000our00ptntour00nt00ntournt ifour0000ntntntntourour0000nt0000our000000avntntour00nt00our00nt00ntavavourourav00000000our00ourou

In [3]:
with torch.no_grad():
    outputs=model.generate(**inputs,streamer=None,
            max_new_tokens=1024,
            do_sample=True,
            temperature=1,
            top_p=0.8)
    outputs

In [4]:
outputs.shape

torch.Size([1, 17])

In [5]:
outputs = outputs[0].cpu().tolist()[len(inputs["input_ids"][0]) :]
response = tokenizer.decode(outputs, skip_special_tokens=True)
response

'，我不能说。\n我是谁，我自己说。<eoa>\n'

In [47]:
outputs

tensor([[    1, 69596, 61275,   345,   262, 60399, 60564, 68502, 60380,   345,
           262, 68687, 86523, 60453,   345,   262, 67621, 62679, 60799, 71952]],
       device='cuda:0')

In [46]:
for _ in range(5):
    outputs=model.generate(**inputs,do_sample=False,num_beams=1)
    print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

/root/.conda/envs/internlm-demo/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['你是谁? 你从哪里来? 你要到哪里去? 这是莎士比亚']
['你是谁? 你从哪里来? 你要到哪里去? 这是莎士比亚']
['你是谁? 你从哪里来? 你要到哪里去? 这是莎士比亚']
['你是谁? 你从哪里来? 你要到哪里去? 这是莎士比亚']
['你是谁? 你从哪里来? 你要到哪里去? 这是莎士比亚']


In [31]:
outputs

tensor([[    1, 69596, 61275,   345,   262, 60399, 60564, 68502, 60380,   345,
           262, 68687, 86523, 60453,   345,   262, 67621, 62679, 60799, 71952]],
       device='cuda:0')

In [17]:
model.chat(tokenizer, "你是谁")

('我是一个名叫书生·浦语的人工智能助手，由上海人工智能实验室开发。我被设计成一个有用、诚实并且无害的智能助手，致力于通过执行常见的基于语言的任务和提供建议来帮助人类。我使用了Transformer模型和深度学习技术，并使用了语言模型作为预训练任务。我能够回答问题、提供定义和解释、将文本从一种语言翻译成另一种语言、总结文本、生成文本、编写故事、分析情感、提供推荐、开发算法、编写代码以及其他任何基于语言的任务。但是我不能看、听、尝、触摸、闻、移动、与物理世界交互、感受情感或体验感官输入、执行需要身体能力的任务。',
 [('你是谁',
   '我是一个名叫书生·浦语的人工智能助手，由上海人工智能实验室开发。我被设计成一个有用、诚实并且无害的智能助手，致力于通过执行常见的基于语言的任务和提供建议来帮助人类。我使用了Transformer模型和深度学习技术，并使用了语言模型作为预训练任务。我能够回答问题、提供定义和解释、将文本从一种语言翻译成另一种语言、总结文本、生成文本、编写故事、分析情感、提供推荐、开发算法、编写代码以及其他任何基于语言的任务。但是我不能看、听、尝、触摸、闻、移动、与物理世界交互、感受情感或体验感官输入、执行需要身体能力的任务。')])

In [ ]:
prompt = "Hey, are you consciours? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=30)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]